In [41]:
import pandas as pd
import numpy as np
import os
from tensorflow import keras
import cv2
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

In [4]:
train_set = pd.read_csv("./train_set.csv")
train_set

,id_img,paths,label
0,0,data\train\angry\0.jpg,angry
1,1,data\train\angry\1.jpg,angry
2,10,data\train\angry\10.jpg,angry
3,10002,data\train\angry\10002.jpg,angry
4,10016,data\train\angry\10016.jpg,angry
...,...,...,...
28816,9969,data\train\surprise\9969.jpg,surprise
28817,9985,data\train\surprise\9985.jpg,surprise
28818,9990,data\train\surprise\9990.jpg,surprise
28819,9992,data\train\surprise\9992.jpg,surprise


In [5]:
test_set = pd.read_csv("./test_set.csv")
test_set

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121
...,...
7061,9806
7062,9830
7063,9853
7064,9878


In [14]:
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
class_names_label

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [11]:
TRAIN_PATH = os.getcwd() + '\\data\\train'
TEST_PATH = os.getcwd() + '\\data\\test'
IMAGE_SIZE = (32,32)

In [15]:
def read_data(path, im_size):
    X = []
    Y = []

    for folder in os.listdir(path):
        label = class_names_label[folder]
        folder_path = os.path.join(path, folder)

    ##### CODE #####
    # Iterar sobre todo lo que haya en path
        for file in os.listdir(folder_path):
            
            # Leer la imagen a color y aplicarle el resize
            image_path = os.path.join(folder_path, file)
            image = imread(image_path)
            smallimage = cv2.resize(image, im_size)
            
            # Guardo en X
            X.append(smallimage)
            Y.append(label)

    return np.array(X), np.array(Y)
    


In [26]:
def read_data_test(path, im_size):
    X = []
    Y = []

    for file in os.listdir(path):
            
        # Leer la imagen a color y aplicarle el resize
        image_path = os.path.join(path, file)
        image = imread(image_path)
        smallimage = cv2.resize(image, im_size)
            
        # Guardo en X
        X.append(smallimage)
        #Y.append(label)

    return np.array(X)
    

In [27]:
X_train, y_train = read_data(TRAIN_PATH, IMAGE_SIZE)
X_test = read_data_test(TEST_PATH, IMAGE_SIZE)

print(X_train.shape)
print(X_test.shape)

(28821, 32, 32)
(7066, 32, 32)


In [23]:
X_train, y_train = shuffle(X_train, y_train, random_state=10)

In [24]:
pd.Series(y_train).value_counts(normalize=True)

3    0.248569
4    0.172860
5    0.171333
2    0.142361
0    0.138545
6    0.111204
1    0.015128
dtype: float64

In [49]:
X_train.shape

(28821, 32, 32)

In [52]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(7, activation='softmax')
])

In [58]:
layers = [
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,3)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(7, activation='sigmoid')
]

model = keras.Sequential(layers)

In [50]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 6, 6, 128)        0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 4608)              0         
                                                                 
 dense_10 (Dense)            (None, 128)              

In [59]:
model.compile(optimizer='adam', loss="sparse_categorical_accuracy", metrics=['accuracy'])

In [46]:
earlystop = EarlyStopping(patience=5)

In [60]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [earlystop]) # batch_size=128,

Epoch 1/20


ValueError: in user code:

    File "C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1050, i

In [ ]:
df_hist = pd.DataFrame(history.history)